# JSON
## Working with nested json

### 1. Examine the file

import people_more_data_json and make a dataframe with these columns:
- Name
- Age
- Street
- City
- Zipcode
- Email
- Phone

In [11]:
import pandas as pd
import json

df = pd.read_json("people_more_data.json")


In [59]:
df.head(2)

,name,age,address,contacts,languages,interests
0,Alice,30,"{'street': '123 Main St', 'city': 'New York', ...","[{'type': 'email', 'value': 'alice@example.com...","[English, Spanish]","{'hobbies': ['Reading', 'Hiking'], 'sports': [..."
1,Bob,25,"{'street': '456 Elm St', 'city': 'Los Angeles'...","[{'type': 'email', 'value': 'bob@example.com'}...","[English, French]","{'hobbies': ['Cooking', 'Gardening'], 'sports'..."


### 2. Separate DataFrames

- Separate the Address column into a separate dataframe
- Separate the Contacts column into a separate dataframe
- Drop all columns except index, name and age from the original dataframe

Tips:
- df2 = df1['column']
- df = df.drop(['columnA','columnB'])

In [63]:
df_address = df['address']
df_address.head(2)

0    {'street': '123 Main St', 'city': 'New York', ...
1    {'street': '456 Elm St', 'city': 'Los Angeles'...
Name: address, dtype: object

In [64]:
df_contacts = df['contacts']
df_contacts.head(2)

0    [{'type': 'email', 'value': 'alice@example.com...
1    [{'type': 'email', 'value': 'bob@example.com'}...
Name: contacts, dtype: object

In [80]:
df=df.drop(columns=['address', 'contacts','languages', 'interests'])

### 3. Loop Address

- use a "for-each loop" to loop through each row in address to get the street, city and zipcode and put the in a separate list
- Create a dictionary with 'street', 'city', and 'zipcode as keys, and the newly created lists as values
- dict_address = {'street' : street, 'city' : city, 'zipcode' : zipcode}
- transform the dictionary into a Dataframe
- merge the newly created dataframe with the orginal one

Tips:
- pd.merge()
- you can access a column, eg 'street' in each row by using row['street'] in the loop


In [30]:
streets = []
cities = []
zipcodes = []

for index, row in df_address.iteritems():
    streets.append(row['address']['street'])
    cities.append(row['address']['city'])
    zipcodes.append(row['address']['zipcode'])

AttributeError: 'Series' object has no attribute 'iteritems'

### Yat's own method

In [69]:
df_address_flattened = pd.json_normalize(df_address)
df_address_flattened.head(5)

,street,city,zipcode
0,123 Main St,New York,10001
1,456 Elm St,Los Angeles,90001
2,789 Oak St,Chicago,60601
3,567 Pine St,San Francisco,94101
4,890 Oakwood Dr,Miami,33101


In [70]:
# df_contacts_flattened.head(5)
df_contacts_flattened = pd.json_normalize(df_contacts)
df_contacts_flattened.head(5)

,0,1
0,"{'type': 'email', 'value': 'alice@example.com'}","{'type': 'phone', 'value': '123-456-7890'}"
1,"{'type': 'email', 'value': 'bob@example.com'}","{'type': 'phone', 'value': '987-654-3210'}"
2,"{'type': 'email', 'value': 'charlie@example.com'}","{'type': 'phone', 'value': '555-123-4567'}"
3,"{'type': 'email', 'value': 'david@example.com'}","{'type': 'phone', 'value': '415-555-1234'}"
4,"{'type': 'email', 'value': 'eva@example.com'}","{'type': 'phone', 'value': '786-555-5678'}"


In [103]:
# Extracting email column from nested data

df_email = df_contacts_flattened[0]
df_email_flattened = pd.json_normalize(df_email)
# df_email_flattened.head(5)

# drop extra column and rename

df_email_flattened.drop(columns="type", inplace=True)
df_email_flattened.rename(columns={"value" : "email"}, inplace=True)
df_email_flattened.head(2)

,email
0,alice@example.com
1,bob@example.com


In [107]:
# Extracting phone column from nested data

df_phone = df_contacts_flattened[1]
df_phone_flattened = pd.json_normalize(df_phone)
df_phone_flattened

# drop 'type' column and rename 'value' to 'phone'

df_phone_flattened.drop(columns='type', inplace=True)
df_phone_flattened.rename(columns={'value' : 'phone'}, inplace=True)

df_phone_flattened.head(5)

,phone
0,123-456-7890
1,987-654-3210
2,555-123-4567
3,415-555-1234
4,786-555-5678


In [81]:
# orginal dataframe

df.head(5)

,name,age
0,Alice,30
1,Bob,25
2,Charlie,35
3,David,28
4,Eva,32


In [111]:
# merge dataframes together

combined = pd.merge(df, df_address_flattened, left_index=True, right_index=True, how = 'left')
combined = pd.merge(combined, df_email_flattened, left_index=True, right_index=True, how = 'left')
combined = pd.merge(combined, df_phone_flattened, left_index=True, right_index=True, how='left')
combined.head(5)

,name,age,street,city,zipcode,email,phone
0,Alice,30,123 Main St,New York,10001,alice@example.com,123-456-7890
1,Bob,25,456 Elm St,Los Angeles,90001,bob@example.com,987-654-3210
2,Charlie,35,789 Oak St,Chicago,60601,charlie@example.com,555-123-4567
3,David,28,567 Pine St,San Francisco,94101,david@example.com,415-555-1234
4,Eva,32,890 Oakwood Dr,Miami,33101,eva@example.com,786-555-5678


In [ ]:
combined = pd.merge(combined, )